# STL file display

This notebook shows how to display STL files used for 3D printing in a notebook.

Displaying relies on the numpy-stl and the ipyvolume libraries.

Upload an STL **ascii** file (*binary files not supported*)

There are some example files you can upload in the [GitHub repository](https://github.com/voila-gallery/render-stl/tree/master/example-stl-meshes).

In [ ]:
import numpy as np
from stl import mesh
from traitlets import dlink
import ipyvolume as ipv
import ipywidgets as widgets
from IPython.display import display
from io import StringIO

# widgets
file_picker = widgets.FileUpload(accept='*.stl')
file_name = widgets.Text()
color_picker = widgets.ColorPicker(
            concise=False,
            description='Pick a color',
            value='white',
            disabled=False)
vbox = widgets.VBox([widgets.HBox([file_picker, file_name]), color_picker])

# display STL file
def show_stl(file_content, color):
    # bridge: FileUpload - stl.mesh
    content = next(iter(file_content.values()))['content']
    io = StringIO()
    io.write(content.decode('utf-8'))
    io.seek(0)
    name, data = mesh.Mesh.load(io)
    m = mesh.Mesh(data, True, name=name, speedups=True)

    ipv.figure()
    ipv.style.use('minimal')
    m2 = ipv.plot_trisurf(m.x.flatten(), m.z.flatten(), m.y.flatten(), triangles=np.arange(m.x.shape[0] * 3).reshape((m.x.shape[0], 3)), color=color)
    return ipv.gcc()

# get filename from FileUpload dict
def get_name(file_upload):
    if len(file_upload) == 0:
        return ''
    return next(iter(file_upload.values()))['metadata']['name']

def change_input(change):
    global vbox, file_picker, file_name, color_picker
    mymesh = show_stl(file_picker.value, color_picker.value)
    vbox.children = (widgets.HBox([file_picker, file_name]), color_picker, mymesh)

# links between widgets
color_picker.observe(change_input, 'value')
dlink((file_picker, 'value'), (file_name, 'value'), get_name)
file_picker.observe(change_input, 'value')

vbox